In [0]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta

In [0]:

# Lista das URLs fornecidas
urls = [
    "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SRAG/2025/INFLUD25-22-12-2025.csv"
]

# Lista para armazenar os DataFrames de cada ano
dfs = []

print("Iniciando leitura dos CSVs...")

for url in urls:
    print(f"Lendo {url}...")
    try:
        # Parâmetros comuns para esses CSVs do Ministério da Saúde
        df_year = pd.read_csv(
            url,
            sep=";",                  # Separador é ponto e vírgula
            encoding="ISO-8859-1",    # Encoding latino (comum em arquivos oficiais brasileiros)
            low_memory=False,         # Evita warnings de tipos mistos
            on_bad_lines="skip"       # Pula linhas corrompidas, se houver
        )
        print(f"  Concluído: {df_year.shape[0]} linhas, {df_year.shape[1]} colunas")
        dfs.append(df_year)
    except Exception as e:
        print(f"  Erro ao ler {url}: {e}")

# Concatena todos os DataFrames
if dfs:
    df = pd.concat(dfs, ignore_index=True)
    print("\nConcatenação concluída!")
    print(f"Dataset total: {df.shape[0]} linhas, {df.shape[1]} colunas")
    print("\nPrimeiras linhas:")
    display(df)
    
else:
    print("Nenhum arquivo foi lido com sucesso.")

In [0]:
from datetime import datetime

print(f"Pré filtro de SRAG: {df.shape[0]} linhas")

# Primeiro, parse das datas relevantes (faça isso logo após carregar os dados)
date_columns = ['DT_SIN_PRI', 'DT_NOTIFIC', 'DT_INTERNA', 'DT_EVOLUCA', 'DT_OBITO', 'DT_NASC', 'DT_PCR', 'DT_ENCERRA']

# Converta TODAS as colunas de data de uma vez, deixando o pandas inferir o formato
for col in date_columns:
    if col in df.columns:
        print(f"Convertendo {col}...")
        df[col] = pd.to_datetime(df[col], errors='coerce')  # Sem 'format=' → infere automaticamente
        nat_count = df[col].isna().sum()
        print(f"  NaT após conversão: {nat_count} (de {len(df)} linhas)")

df = df.copy()

# ETAPA 4: CLASSIFICAÇÃO DE VÍRUS EM CASOS E ÓBITOS (fazemos antes da 3, pois precisamos de covid_caso e covid_obito)
# COVID-19
df['covid_caso'] = ((df['PCR_SARS2'] == 1) | (df['AN_SARS2'] == 1) | (df['CLASSI_FIN'] == 5)).astype(int)
df['covid_obito'] = ((df['covid_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# INFLUENZA A - H1N1
df['influenza_h1n1_caso'] = (df['PCR_FLUASU'] == 1).astype(int)
df['influenza_h1n1_obito'] = ((df['influenza_h1n1_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# INFLUENZA A - H3N2
df['influenza_h3n2_caso'] = (df['PCR_FLUASU'] == 2).astype(int)
df['influenza_h3n2_obito'] = ((df['influenza_h3n2_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# INFLUENZA A - Não Subtipável
df['influenza_a_n_subtipavel_caso'] = (df['PCR_FLUASU'] == 4).astype(int)
df['influenza_a_n_subtipavel_obito'] = ((df['influenza_a_n_subtipavel_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# INFLUENZA A - Não Subtipada
df['influenza_a_n_sub_caso'] = (
    (df['PCR_FLUASU'] == 3) |
    (((df['TP_FLU_AN'] == 1) | (df['TP_FLU_PCR'] == 1)) &
     (df['influenza_h1n1_caso'] == 0) & (df['influenza_h3n2_caso'] == 0) & (df['influenza_a_n_subtipavel_caso'] == 0))
).astype(int)
df['influenza_a_n_sub_obito'] = ((df['influenza_a_n_sub_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# INFLUENZA A – Total
df['influenza_a_total'] = (
    (df['influenza_a_n_sub_caso'] == 1) | (df['influenza_h1n1_caso'] == 1) |
    (df['influenza_h3n2_caso'] == 1) | (df['influenza_a_n_subtipavel_caso'] == 1)
).astype(int)
df['influenza_a_total_obito'] = ((df['influenza_a_total'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# INFLUENZA B – Victoria
df['influenza_b_vict_caso'] = (df['PCR_FLUBLI'] == 1).astype(int)
df['influenza_b_vict_obito'] = ((df['influenza_b_vict_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# INFLUENZA B – Yamagata
df['influenza_b_yam_caso'] = (df['PCR_FLUBLI'] == 2).astype(int)
df['influenza_b_yam_obito'] = ((df['influenza_b_yam_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# INFLUENZA B – Inconclusivo (não subtipado)
df['influenza_b_inconclusivo_caso'] = (
    ((df['TP_FLU_AN'] == 2) | (df['TP_FLU_PCR'] == 2)) &
    (df['influenza_b_vict_caso'] == 0) & (df['influenza_b_yam_caso'] == 0)
).astype(int)
df['influenza_b_inconclusivo_obito'] = ((df['influenza_b_inconclusivo_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# INFLUENZA B – Total
df['influenza_b_total'] = (
    (df['influenza_b_vict_caso'] == 1) | (df['influenza_b_yam_caso'] == 1) |
    (df['influenza_b_inconclusivo_caso'] == 1)
).astype(int)
df['influenza_b_total_obito'] = ((df['influenza_b_total'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# VSR (Vírus Sincicial Respiratório)
df['vsr_caso'] = ((df['AN_VSR'] == 1) | (df['PCR_VSR'] == 1)).astype(int)
df['vsr_obito'] = ((df['vsr_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# Outros Vírus Respiratórios (OVR)
# Incluindo Parainfluenza 1-4, Adenovírus, Bocavírus, Metapneumovírus, Rinovírus e outros
ovr_vars_pcr = ['PCR_PARA1', 'PCR_PARA2', 'PCR_PARA3', 'PCR_PARA4', 'PCR_ADENO', 'PCR_BOCA', 'PCR_METAP', 'PCR_RINO', 'PCR_OUTRO']
ovr_vars_an = ['AN_PARA1', 'AN_PARA2', 'AN_PARA3', 'AN_PARA4', 'AN_ADENO', 'AN_BOCA', 'AN_METAP', 'AN_OUTRO']  # Ajuste se houver AN_RINO etc.
df['ovr_caso'] = 0
for var in ovr_vars_pcr + ovr_vars_an:
    if var in df.columns:
        df['ovr_caso'] = df['ovr_caso'] | (df[var] == 1)
df['ovr_caso'] = df['ovr_caso'].astype(int)
df['ovr_obito'] = ((df['ovr_caso'] == 1) & (df['EVOLUCAO'] == 2)).astype(int)

# ETAPA 5: CLASSIFICAÇÃO DE VÍRUS EM CASOS E ÓBITOS SEM CODETECÇÕES
# Identificar codetecções: mais de um vírus positivo
virus_caso_cols = ['covid_caso', 'influenza_a_total', 'influenza_b_total', 'vsr_caso', 'ovr_caso']
df['num_virus'] = df[virus_caso_cols].sum(axis=1)
df_no_co = df[df['num_virus'] <= 1].copy()  # Apenas casos com 0 ou 1 vírus (exclui codetecções)

print(f"Pós filtro de SRAG: {df.shape[0]} linhas")

# Verificação rápida
print("\nContagem de casos por vírus (com codetecções):")
print(df[virus_caso_cols].sum())

print("\nContagem de casos por vírus (sem codetecções):")
print(df_no_co[virus_caso_cols].sum())

In [0]:
# Encontre a data de referência (última disponível)
data_ref = df['DT_SIN_PRI'].max()
print(f"Data de referência: {data_ref.date()}")

# Filtre dados recentes para métricas e gráficos
df_recent = df[df['DT_SIN_PRI'] <= data_ref].copy()

# Calcule total de casos e óbitos
total_casos = len(df_recent)
total_obitos = len(df_recent[df_recent['EVOLUCAO'] == 2])  # Evolução == 2 é óbito

In [0]:
# 1. Taxa de aumento de casos (ex.: crescimento semanal, últimos 7 dias vs. 7 dias anteriores)
ultimos_7_dias_inicio = data_ref - timedelta(days=6)
anteriores_7_dias_inicio = data_ref - timedelta(days=13)
anteriores_7_dias_fim = data_ref - timedelta(days=7)

casos_ultimos_7 = len(df_recent[(df_recent['DT_SIN_PRI'] >= ultimos_7_dias_inicio) & (df_recent['DT_SIN_PRI'] <= data_ref)])
casos_anteriores_7 = len(df_recent[(df_recent['DT_SIN_PRI'] >= anteriores_7_dias_inicio) & (df_recent['DT_SIN_PRI'] <= anteriores_7_dias_fim)])

if casos_anteriores_7 > 0:
    taxa_aumento = ((casos_ultimos_7 - casos_anteriores_7) / casos_anteriores_7) * 100
else:
    taxa_aumento = 0  # Evitar divisão por zero
print(f"Taxa de aumento de casos (últimos 7 dias vs. anteriores): {taxa_aumento:.2f}%")

# 2. Taxa de mortalidade (letalidade: óbitos / casos)
taxa_mortalidade = (total_obitos / total_casos * 100) if total_casos > 0 else 0
print(f"Taxa de mortalidade (geral no dataset): {taxa_mortalidade:.2f}%")

# 3. Taxa de ocupação de UTI (proxy: % de casos que foram para UTI; dados reais de leitos não disponíveis no df)
if 'UTI' in df_recent.columns:
    casos_uti = len(df_recent[df_recent['UTI'] == 1])
    taxa_uti = (casos_uti / total_casos * 100) if total_casos > 0 else 0
    print(f"Taxa de ocupação de UTI (proxy: % de casos em UTI): {taxa_uti:.2f}% (Aviso: Não reflete % de leitos disponíveis; busque em boletins Fiocruz para dados reais)")
else:
    print("Coluna 'UTI' não encontrada; taxa não calculada.")

# 4. Taxa de vacinação da população (proxy: % de casos vacinados; não populacional)
if 'VACINA_COV' in df_recent.columns:
    vacinados = len(df_recent[df_recent['VACINA_COV'] == 1])
    taxa_vacinacao = (vacinados / total_casos * 100) if total_casos > 0 else 0
    print(f"Taxa de vacinação (proxy: % de casos com vacina COVID): {taxa_vacinacao:.2f}% (Aviso: Representa apenas hospitalizados; taxa populacional real é ~80-90% para esquema completo em 2023-2024, sem dados confirmados para 2025)")
else:
    print("Coluna 'VACINA_COV' não encontrada; taxa não calculada.")

In [0]:
# Criar DataFrame separado com as métricas
metrics_data = {
    'metrica': [
        'taxa_aumento_casos_7d',
        'taxa_mortalidade',
        'taxa_uti_proxy',
        'taxa_vacinacao_proxy'
    ],
    'valor': [
        taxa_aumento,
        taxa_mortalidade,
        taxa_uti,
        taxa_vacinacao
    ],
    'descricao': [
        'Taxa de aumento de casos nos últimos 7 dias vs. 7 dias anteriores (%)',
        'Taxa de mortalidade geral (óbitos por SRAG / total casos) (%)',
        'Proxy para taxa de ocupação de UTI (% de casos internados em UTI). Nota: Não reflete leitos disponíveis reais.',
        'Proxy para taxa de vacinação (% de casos vacinados contra COVID). Nota: Baseado em hospitalizados; taxa populacional real é maior.'
    ],
    'data_referencia': [data_ref] * 4
}

metrics_df = pd.DataFrame(metrics_data)

# Exibir o DataFrame
display(metrics_df)

metrics_df = spark.createDataFrame(metrics_df) 
# Salvar como tabela Delta (substitua "nome_da_tabela" pelo desejado, ex: "srag_metrics_2025")
metrics_df.write.format("delta").mode('overwrite').option("overwriteSchema", "True").saveAsTable("srag_metrics")

In [0]:
# Últimos 30 dias
inicio_30_dias = data_ref - timedelta(days=29)
df_30_dias = df_recent[df_recent['DT_SIN_PRI'] >= inicio_30_dias].copy()

# Agrupar por dia
casos_diarios = df_30_dias.groupby(df_30_dias['DT_SIN_PRI'].dt.date).size().reset_index(name='Casos')
casos_diarios['DT_SIN_PRI'] = pd.to_datetime(casos_diarios['DT_SIN_PRI'])

# Plot
plt.figure(figsize=(12, 6))
plt.plot(casos_diarios['DT_SIN_PRI'], casos_diarios['Casos'], marker='o')
plt.title('Número Diário de Casos de SRAG (Últimos 30 Dias)')
plt.xlabel('Data')
plt.ylabel('Número de Casos')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [0]:
# Últimos 12 meses
inicio_12_meses = data_ref - timedelta(days=365)  # Aproximado
df_12_meses = df_recent[df_recent['DT_SIN_PRI'] >= inicio_12_meses].copy()

# Agrupar por mês
df_12_meses['Mes'] = df_12_meses['DT_SIN_PRI'].dt.to_period('M')
casos_mensais = df_12_meses.groupby('Mes').size().reset_index(name='Casos')
casos_mensais['Mes'] = casos_mensais['Mes'].dt.to_timestamp()

# Plot
plt.figure(figsize=(12, 6))
plt.bar(casos_mensais['Mes'], casos_mensais['Casos'], width=20)  # Largura aproximada para meses
plt.title('Número Mensal de Casos de SRAG (Últimos 12 Meses)')
plt.xlabel('Mês')
plt.ylabel('Número de Casos')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()